In [11]:
!pip install git+https://github.com/huggingface/transformers
# !pip install cloud-tpu-client==0.10 torch==1.13.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl
!pip install wandb
!pip install evaluate

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\andre\AppData\Local\Temp\pip-req-build-dcj1apwe'



  Cloning https://github.com/huggingface/transformers to c:\users\andre\appdata\local\temp\pip-req-build-dcj1apwe
  Resolved https://github.com/huggingface/transformers to commit d994473b05a83ea398d9f10ca458855df095e22d
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 81.4/81.4 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 132.9/132.9 kB ? eta 0:00:00
     -------------------------------------- 451.7/451.7 kB 7.1 MB/s eta 0:00:00
     -------------------------------------- 139.5/139.5 kB 4.2 MB/s eta 0:00:00
     -------------------------------------- 323.5/323.5 kB 6.7 MB/s eta 0:00:00
     --------------------------------------- 20.3/20.3

In [2]:
# !pip install torch transformers wandb -qqq

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            TextDataset, DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
from transformers import EarlyStoppingCallback
# from huggingface_hub.hf_api import HfAp
# import api.filter


import os
import numpy as np
import re
import pandas as pd
import tensorflow as tf
import torch
import pathlib
import random
import evaluate



import wandb
wandb.init(project="my-awesome-project")

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: qulxis. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [5]:
#Tokenize:
file_path =  "data/special_token_versions/textfiles/train_data.txt"  

file_path_val = "data/special_token_versions/textfiles/test_data.txt" # use test_data.txt for full scale
use_special_tokens = True # use custom tokens
special_tokens_manual= ['<|sensander|>', '<|paddingtonbear|>', '<|hankgreen|>', '<|joerogan|>', '<|elonmusk|>', '<|polite|>', '<|impolite|>','<|neutral|>']
#read new tokens:
token_file_path = "data/special_token_versions/keys.csv"
new_special_tokens = pd.read_csv(token_file_path)["Keys"].values.tolist() + special_tokens_manual #add these to special tokens
special_tokens_dict = {'additional_special_tokens': new_special_tokens} # use for direct add
wandb.run.name = file_path
"""
Here, we specify what we train we use to train our data. In our case, we
have several special tokens (ones that shouldn't be split). 

We include subject tokens, user tokens, and politeness tokens manually in special_tokens_manual

"""

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

#add special tokens:
if use_special_tokens:
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict) #adds special tokens
    model.resize_token_embeddings(len(tokenizer))


block_size = tokenizer.model_max_length
train_dataset = TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=block_size, overwrite_cache=True)
evaluation_dataset = TextDataset(tokenizer=tokenizer, file_path=file_path_val, block_size=block_size, overwrite_cache=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#setup wanbi:
%env WANDB_PROJECT=tweet_analysis

wandb.run.name = file_path
wandb.run.save()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\andre/.cache\huggingface\hub\models--gpt2\snapshots\f27b190eeac4c2302d24068eabf5e9d6044389ae\config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
  

env: WANDB_PROJECT=tweet_analysis


True

In [6]:
print(len(new_special_tokens))

22246


In [7]:
!nvidia-smi

Tue Dec 13 23:29:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.98       Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8    N/A /  N/A |    184MiB /  2048MiB |     21%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
import evaluate

ModuleNotFoundError: No module named 'evaluate'

In [8]:

#Train:
# START: COPIED FROM <https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG- >


ALLOW_NEW_LINES = False     # seems to work better <--- from source
LEARNING_RATE = 1.372e-4
EPOCHS = 4
seed = random.randint(0,2**32-1)
# END: COPIED FROM <https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG- >


def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



training_args = TrainingArguments(
    report_to="wandb",
    output_dir="./model_files",
    overwrite_output_dir=True,
    do_train=True,
    evaluation_strategy = 'steps',# num_train_epochs=1, #new
    eval_steps = 5000, #new
    per_device_train_batch_size=1,
    prediction_loss_only=True,
    logging_steps=5,
    save_steps=0,
    seed=seed,
    learning_rate = LEARNING_RATE,
    metric_for_best_model = 'f1',#new
    load_best_model_at_end = True, #new
    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset = evaluation_dataset,
    compute_metrics=compute_metrics, #new
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]) #new

# START: COPIED FROM <https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG- >

#LR schedule stuff?
train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader)
trainer.create_optimizer_and_scheduler(num_train_steps)
trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps)
# END: COPIED FROM <https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG- >



PyTorch: setting up devices
c:\Users\andre\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
metric = evaluate.load("glue", "mrpc")


***** Running training *****
  Num examples = 2496
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 7488
  Number of trainable parameters = 141521664
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


  0%|          | 0/7488 [00:00<?, ?it/s]

{'loss': 51.805, 'learning_rate': 0.00013719864155118674, 'epoch': 0.0}


KeyboardInterrupt: 

In [9]:
trainer.model.config.task_specific_params['text-generation'] = {
    'do_sample': True,
    'min_length': 15,
    'max_length': 100,
    'temperature': 100,
    'top_p': 0.95,
    'prefix': '<|endoftext|>'}

In [10]:
#Save Model:
trainer.save_model()

Saving model checkpoint to ./model_files
Configuration saved in ./model_files/config.json
Model weights saved in ./model_files/pytorch_model.bin
tokenizer config file saved in ./model_files/tokenizer_config.json
Special tokens file saved in ./model_files/special_tokens_map.json


In [11]:
a = trainer.state.log_history
print(a[0])

{'loss': 2.9403, 'learning_rate': 0.0001371792092297936, 'epoch': 0.01, 'step': 5}


# Predict:


In [20]:
# prepare input
predictions = []
tag = "<|elonmusk|>"
polite_tag = "<|neutral|>"
topic1 = '<|spacex|>'
topic2 = '<|failure|>'
start = ""

"""
Notes:
Temperature: trade-off between variety and politeness clarity
Beam Search vs Top-k/Top-p. Beam search is a lot more coherent with a trade off for variety.

Options:

Naive Beam Search: Num_beams = 10, all else off
Top K with Nucleus Sampling: top_p = 0.95, top_k = 10-20, do_sample=True
Beam-search multinomial sampling : Num_beams = 10 + do_sample = True
Diverse beam-search decoding: Num_beams = 10 + num_beam_groups = 2





"""

start_with_bos = '<|endoftext|>'+tag+polite_tag+topic1+topic2 + start
encoded_prompt = trainer.tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)

# prediction
output_sequences = trainer.model.generate(
    input_ids=encoded_prompt,
    max_length=160, #originally 160
    min_length=10, #originally 10
    temperature = 1, #originally 1
    no_repeat_ngram_size=2,    
    
    
    #Edit stuff here down to change decoding 
# BEAM
    # num_beams=10, #on or off

# TOP    
    do_sample=True, # for multinomial beam search and top sampling
    top_p = 0.95, #0.95
    top_k = 50, #10-20

# MULTINOMIAL
    # num_beams=10, #on or off
    # do_sample=True, # for multinomial beam search and top sampling

# DIVERSE 
    # num_beams=10, #on or off  
    # num_beam_groups = 2, # on or off, must be a multiple of num_beams

    
    
    num_return_sequences= 10, #must = num_beam for diverse beam-search
    
    )
# START: COPIED FROM <https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG- >

generated_sequences = []

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = trainer.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    if not ALLOW_NEW_LINES:
        limit = text.find('\n')
        text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

for i, g in enumerate(generated_sequences):
    predictions.append([start, g])
# END: COPIED FROM <https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG- >


for pair in predictions:
  print(pair[1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/ecbm4040/envTF24/lib/python3.6/site-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect. "


Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I'll get a megaphone like Mr Curry's to remind everyone.
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I suggest preparing some marmalade sandwiches for an extra special elevenses!
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I suggest preparing some marmalade sandwiches for an extra special elevenses!
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I'll get a megaphone like Mr Curry's to remind everyone.
Mr. Brown, I'm very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I'll get a meg megaphone like Mr Curry's to remind everyone.
Mr. Brown, I’m very pleased that you would like me to send you a reminder about my new film #Paddington2. I think I

In [13]:
print("warning")

warning


In [21]:
username = "paddingtonbear" #twitter user
type_tweet = "neutral" #polite, impolite, neutral
decoder = "diverse" #beam, top (top_k + top_p), multinomial (Num_beams + do_sample), diverse (Num_beams + num_beam_groups)

import pandas as pd
df = pd.DataFrame(columns = ["Target","Prompt","Tweets","Type"])
target_col = [username]*len(predictions) # target col
type_col = [type_tweet]*len(predictions) # type col
prompt_col = []
tweets_col = []

for pair in predictions:
    prompt_col.append(pair[0])
    tweets_col.append(pair[1])

df["Target"] = target_col
df["Prompt"] = prompt_col
df["Tweets"] = tweets_col
df["Type"] = type_col
df.reset_index()
# print(df)

df.to_csv('responses/{}_{}_{}.csv'.format(username,type_tweet,decoder), index=False)
#check writing:

df_test = pd.read_csv('responses/{}_{}_{}.csv'.format(username,type_tweet,decoder))
print(df_test)



           Target      Prompt  \
0  paddingtonbear  Mr. Brown,   
1  paddingtonbear  Mr. Brown,   
2  paddingtonbear  Mr. Brown,   
3  paddingtonbear  Mr. Brown,   
4  paddingtonbear  Mr. Brown,   
5  paddingtonbear  Mr. Brown,   
6  paddingtonbear  Mr. Brown,   
7  paddingtonbear  Mr. Brown,   
8  paddingtonbear  Mr. Brown,   
9  paddingtonbear  Mr. Brown,   

                                              Tweets     Type  
0  Mr. Brown, I'm very pleased that you would lik...  neutral  
1  Mr. Brown, I'm very pleased that you would lik...  neutral  
2  Mr. Brown, I'm very pleased that you would lik...  neutral  
3  Mr. Brown, I'm very pleased that you would lik...  neutral  
4  Mr. Brown, I'm very pleased that you would lik...  neutral  
5  Mr. Brown, I’m very pleased that you would lik...  neutral  
6  Mr. Brown, I’m very pleased that you would lik...  neutral  
7  Mr. Brown, I'm very pleased that you would lik...  neutral  
8  Mr. Brown, I'm very pleased that you would lik...  neutra

Empty DataFrame
Columns: [Unnamed: 0, Target, Tweets, Type]
Index: []


# New Section
